# write ELFI wrapper for our simulators and run

- runs on M/G/1 model, but still returns utter nonsense as samples !
- just bad outcome or used wrong?

In [ ]:
import scipy.stats as ss
import numpy as np
import elfi

from snl.util.plot import plot_hist_marginals

seed = 42    

model_id = 'mg1'

if model_id == 'gauss':
    from lfimodels.snl_exps.util import load_setup_gauss as load_setup
    from lfimodels.snl_exps.util import load_gt_gauss as load_gt
    from lfimodels.snl_exps.util import init_g_gauss as init_g
    from lfimodels.snl_exps.util import draw_sample_uniform_prior_33 as fast_sampler    
    

    theta1 = elfi.Prior('uniform', -3, 3)
    theta2 = elfi.Prior('uniform', -3, 3)
    theta3 = elfi.Prior('uniform', -3, 3)
    theta4 = elfi.Prior('uniform', -3, 3)
    theta5 = elfi.Prior('uniform', -3, 3)
    
    thetas = [theta1, theta2, theta3, theta4, theta5]
    
    g = init_g(seed=seed)
    def simulator(theta1, theta2, theta3, theta4, theta5, batch_size=1, random_state=None):
        theta1, theta2, theta3, theta4, theta5= np.atleast_1d(theta1, theta2, theta3, theta4, theta5)
        theta = np.array([theta1, theta2, theta3, theta4, theta5]).T
        return g.model.gen(theta, n_reps=batch_size)
    
    bounds={'theta1':(-3,3), 'theta2':(-3,3), 'theta3':(-3,3), 'theta4':(-3,3), 'theta5':(-3,3)}        
elif model_id == 'lv':
    from lfimodels.snl_exps.util import load_setup_lv as load_setup
    from lfimodels.snl_exps.util import load_gt_lv as load_gt
    from lfimodels.snl_exps.util import init_g_lv as init_g
    from lfimodels.snl_exps.util import draw_sample_uniform_prior_52 as fast_sampler
    

    theta1 = elfi.Prior('uniform', -5, 2)
    theta2 = elfi.Prior('uniform', -5, 2)
    theta3 = elfi.Prior('uniform', -5, 2)
    theta4 = elfi.Prior('uniform', -5, 2)
    thetas = [theta1, theta2, theta3, theta4]
    
    g = init_g(seed=seed)
    def simulator(theta1, theta2, theta3, theta4, batch_size=1, random_state=None):
        theta1, theta2, theta3, theta4= np.atleast_1d(theta1, theta2, theta3, theta4)
        theta = np.array([theta1, theta2, theta3, theta4]).T
        return g.model.gen(theta, n_reps=batch_size)
    
    bounds={'theta1':(-5,2), 'theta2':(-5,2), 'theta3':(-5,2), 'theta4':(-5,2)}    
    
elif model_id == 'mg1':
    from lfimodels.snl_exps.util import load_setup_mg1 as load_setup
    from lfimodels.snl_exps.util import load_gt_mg1 as load_gt
    from lfimodels.snl_exps.util import init_g_mg1 as init_g
    from lfimodels.snl_exps.util import draw_sample_shifted_uniform_prior as fast_sampler
    
    theta1 = elfi.Prior('uniform', 0, 10)
    theta2 = elfi.Prior('uniform', 0, 20)
    theta3 = elfi.Prior('uniform', 0, 1./3.)
    thetas = [theta1, theta2, theta3]
    
    g = init_g(seed=seed)
    def simulator(theta1, theta2, theta3, batch_size=1, random_state=None):
        theta1, theta2, theta3 = np.atleast_1d(theta1, theta2, theta3)
        theta = np.array([theta1, theta2, theta3]).T
        return g.model.gen(theta, n_reps=batch_size)
    
    bounds={'theta1':(0,10), 'theta2':(0, 20), 'theta3':(0, 1./3)}

    
pars_true, obs_stats = load_gt(generator=init_g(seed=seed))
print('pars_true : ', pars_true)
print('obs_stats : ', obs_stats)


import scipy.stats as ss
import numpy as np


# Generate some data (using a fixed seed here)
np.random.seed(20170525)
y0 = simulator(*pars_true)
print(y0)

# Add the simulator node and observed data to the model
sim = elfi.Simulator(simulator, *thetas, observed=y0)

# Add summary statistics to the model
ss = []
for i in range(g.summary.n_summary):
    fun = lambda x: g.summary.calc([np.asarray(x).flatten()])[:,i]
    ss.append( elfi.Summary(fun, sim) )

# Specify distance as euclidean between summary vectors (S1, S2) from simulated and
# observed data
d = elfi.Distance('euclidean', *ss)

log_d = elfi.Operation(np.log, d)

bolfi = elfi.BOLFI(log_d, batch_size=1, initial_evidence=20, update_interval=10,
                   bounds=bounds, acq_noise_var=[0.1 for i in range(len(thetas))], seed=seed)

%time post = bolfi.fit(n_evidence=100)

%time result_BOLFI = bolfi.sample(100, info_freq=1000)
result_BOLFI.plot_traces();

bo_samples = result_BOLFI.samples
fig =plot_hist_marginals(np.vstack([bo_samples[key] for key in bo_samples.keys()]).T)
fig.show()